In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import pickle
import scipy.stats as st
from numpy import sqrt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score, average_precision_score
from catboost import CatBoostClassifier
%matplotlib inline

In [17]:
#Загрузка датасета
ds = pd.read_excel('dataset_2.xlsx')
for i in range(4513):
    ds['average_eval'][i] = ds['average_eval'][i].replace(',', '.')
    #ds['average_eval'][i] = float(ds['average_eval'][i])
#ds = ds[ds['stud_kind']=='Технические']
#Настройка классов
#ds = ds[(ds['average_eval'] >= 5.0) | (ds['average_eval'] <= 3.5)]
ds['class'] = ds['average_eval'].apply(lambda x: 1 if float(x)>=4.2 else 0)
#ds['class'] = ds['status'].apply(lambda x: 0 if (x=='Отчислен') else 1)
#ds['class'] = ds['average_eval']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
ds['class']

0       1
1       1
2       1
3       1
4       0
       ..
4508    0
4509    0
4510    0
4511    1
4512    0
Name: class, Length: 4513, dtype: int64

In [19]:
ds['status'].unique()

array(['Отчислен', 'Является студентом',
       'Находится в академическом отпуске', 'Является выпускником'],
      dtype=object)

In [20]:
ds = ds[(ds['status'] != 'Отчислен') & (ds['status'] != 'Находится в академическом отпуске')]

In [21]:
# Строки с подписками превратим в список подписок 
ds.publics = ds.publics.str.split(',')
# разделим датасет на два, один - для мальчиков, второй - для девочек 
ds_male=ds.query('sex==2')
ds_female=ds.query('sex==1')
ds_male = ds_male.drop(columns = ['sex']) 
ds_female = ds_female.drop(columns = ['sex']) 
print('мальчиков в датасете - {}\nдевочек - {}'.format(ds_male.shape[0], ds_female.shape[0])) 

print('Мальчики:\n{}\nДевочки:\n{}'.format(ds_male['class'].value_counts(), ds_female['class'].value_counts()))


мальчиков в датасете - 1648
девочек - 2568
Мальчики:
1    840
0    808
Name: class, dtype: int64
Девочки:
1    1816
0     752
Name: class, dtype: int64


In [22]:
n_boys = ds_male.shape[0]

In [23]:
ds_male['class'].value_counts()

1    840
0    808
Name: class, dtype: int64

In [24]:
# преобразуем подписки в матрицу бинарных признаков
mlb = MultiLabelBinarizer()
matrix_ds_male=pd.DataFrame(mlb.fit_transform(ds_male.publics), columns=mlb.classes_, index = ds_male.index)
print(matrix_ds_male.shape)
X = matrix_ds_male.loc[:,:]
y = ds_male['class']
matrix_ds_male

(1648, 42657)


,'100000694','100008198','100016590','100021690','100029019','100029181','100039832','100040520','100043477','100044564',...,['98169847',['98210264',['98223110',['98808011',['9884911',['98886401',['99009078',['99255023',['99391270',['99776591'
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#Точный тест Фишера
array_of_significant_communities = []
for i in range(n_boys):
    #ct - таблица сопряженности
    ct = pd.crosstab(y, X.transpose().iloc[i])
    oddsratio, pvalue = st.fisher_exact(ct)
    if pvalue < 0.05:
        print(mlb.classes_[i])
        array_of_significant_communities .append(mlb.classes_[i])
        print(ct)
        print("Exact Fisher odds ratio: " + str(oddsratio))
        print("Exact Fisher p-value: " + str(pvalue) + "\n")
#Pvalue < 0.05 - отвергаем гипотезу H0 о независимости между успеваемостью студента и наличием/отсутствием его
#подписки на сообщество(паблик ВК) с данным номером

 '100249506'
 '100249506'    0  1
class               
0             807  1
1             831  9
Exact Fisher odds ratio: 8.740072202166065
Exact Fisher p-value: 0.021342777800768743

 '100419172'
 '100419172'    0   1
class                
0             782  26
1             827  13
Exact Fisher odds ratio: 0.4727932285368803
Exact Fisher p-value: 0.03395655463427151

 '102251259'
 '102251259'    0  1
class               
0             801  7
1             839  1
Exact Fisher odds ratio: 0.13638685509960838
Exact Fisher p-value: 0.035283089915655694

 '103319735'
 '103319735'    0  1
class               
0             808  0
1             834  6
Exact Fisher odds ratio: inf
Exact Fisher p-value: 0.031142176351709624

 '103663851'
 '103663851'    0  1
class               
0             802  6
1             840  0
Exact Fisher odds ratio: 0.0
Exact Fisher p-value: 0.013759463337177934

 '104144232'
 '104144232'    0  1
class               
0             800  8
1             840  0
Exact

In [29]:
array_of_significant_communities 

[" '100249506'",
 " '100419172'",
 " '102251259'",
 " '103319735'",
 " '103663851'",
 " '104144232'",
 " '105527557'",
 " '105693388'",
 " '105694313'",
 " '105988524'",
 " '106036897'",
 " '106188044'",
 " '106600980'",
 " '107232016'",
 " '107714227'",
 " '108921639'"]

In [33]:
# разделим набор данных на обучающую и тестовую выборки, размер тестовой выборки - 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
#удалим паблики на которые во всем X_train подписаны менее 10 человек
count_person_train = X_train.shape[0] - 10
X_train = X_train.loc[:, (X_train == 0).sum() <= count_person_train] 
print(X_train.shape)
# Оставим в X_test только me сообщества, которые есть в X_train
X_test = X_test[X_train.columns.tolist()]
print(X_test.shape)

(1236, 1663)
(412, 1663)


In [34]:
#Подсчет весов для сообществ
w_ds = pd.concat([X_train, y_train], axis=1,sort=False)
lst_w = [] 
lst_n = [] 
lst_n_0 = [] 
lst_n_1 = [] 
lst_n_2 = []
for i in range(w_ds.shape[1] - 1):
    lst_w.append((sum((w_ds['class']==0) & (w_ds.iloc[:, i]==1))*(-1) / sum(0+w_ds.iloc[:,i]==1) + \
                  sum((w_ds['class']==1) & (w_ds.iloc[:,i]==1))*(1) / sum(0+w_ds.iloc[:,i]==1)))
    lst_n.append(sum(w_ds.iloc[:,i]==1))
    lst_n_0.append(sum((w_ds['class']==0) & (w_ds.iloc[:,i]==1)))
    lst_n_1.append(sum((w_ds['class']==1) & (w_ds.iloc[:,i]==1)))
lst_w
                                                                                       
lst_w
#Записать веса сообществ в отдельный датафрейм
ds_lst_w = pd.DataFrame([lst_w, lst_n, lst_n_0, lst_n_1], columns=X_train.columns.tolist(),\
                       index=['w', 'n', 'n_0', 'n_1']).transpose()
ds_lst_w['n'] = ds_lst_w['n'].apply(lambda x: int(x))
ds_lst_w['n_0'] = ds_lst_w['n_0'].apply(lambda x: int(x))
ds_lst_w['n_1'] = ds_lst_w['n_1'].apply(lambda x: int(x))

In [35]:
lst_w

[-0.6363636363636365,
 -0.40740740740740744,
 -0.5,
 -0.28571428571428575,
 -0.06666666666666665,
 0.39999999999999997,
 -0.4285714285714286,
 0.16666666666666669,
 -0.16666666666666669,
 0.03658536585365857,
 0.08571428571428569,
 0.3333333333333333,
 -0.08235294117647057,
 -0.22222222222222227,
 -0.22222222222222227,
 -0.30000000000000004,
 -0.39999999999999997,
 0.2727272727272727,
 -0.2727272727272727,
 -0.3846153846153846,
 0.06666666666666665,
 0.0,
 0.05555555555555558,
 0.2727272727272727,
 -0.23076923076923078,
 0.11111111111111116,
 0.07692307692307687,
 0.3333333333333333,
 0.23076923076923078,
 -0.09090909090909088,
 0.14285714285714285,
 -0.6666666666666667,
 -0.16666666666666669,
 -0.2727272727272727,
 -0.5384615384615385,
 -0.625,
 -0.17241379310344823,
 0.19999999999999996,
 -0.07692307692307687,
 -0.19999999999999996,
 0.6666666666666667,
 -0.19999999999999996,
 -0.2631578947368421,
 0.0,
 -0.2727272727272727,
 0.10000000000000003,
 0.07692307692307687,
 0.076923076923

In [36]:
import pickle
#сохранить веса в sav
filename = r'{}.sav'.format('ds_male')
pickle.dump(ds_lst_w.transpose(), open(filename, 'wb'))

In [37]:
#Добавить веса в наборах Х_train
#каждый коэффициент на свой столбец
for i in range(X_train.shape[1]):
    X_train.iloc[:,i] = X_train.iloc[:,i].astype(float) * lst_w[i]
    #X_train.iloc[:,i] = X_train.iloc[:,i].astype(float) * 1
#Добавить веса в X_test
for i in range(X_test.shape[1]):
    X_test.iloc[:,i] = X_test.iloc[:,i].astype(float) * lst_w[i]
    #X_test.iloc[:,i] = X_test.iloc[:,i].astype(float) * 1


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score, precision_score
from catboost import CatBoostClassifier
#LOGISTIC REGRESSION
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
LR_accuracy, LR_precision = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred) 
LR_recall, LR_f1 = recall_score(y_test, y_pred), f1_score(y_test, y_pred)
print('Мальчики:')
print('Логистическая регрессия:\n accuracy = {} \n precision = {} \n recall = {} \n f1 = {}'.format(LR_accuracy, LR_precision, LR_recall, LR_f1))
print(roc_auc_score(y_test, y_pred))
#print(average_precision_score(y_test, logreg.predict_proba(X_test)[:,1]))

Мальчики:
Логистическая регрессия:
 accuracy = 0.5849514563106796 
 precision = 0.5802469135802469 
 recall = 0.6714285714285714 
 f1 = 0.6225165562913907
0.5832390381895332


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [39]:
#CatBoost
model = CatBoostClassifier(verbose=False, learning_rate = 0.01)
model.fit(X_train, y_train)
cat_y_pred = model.predict(X_test)
CB_accuracy, CB_precision = accuracy_score(y_test, cat_y_pred), precision_score(y_test, cat_y_pred) 
CB_recall, CB_f1 = recall_score(y_test, cat_y_pred), f1_score(y_test, cat_y_pred)
print('Мальчики:')
print('CatBoost: \n accuracy = {} \n precision = {} \n recall = {} \n f1 = {}'.format(CB_accuracy, CB_precision, CB_recall, CB_f1))
print(roc_auc_score(y_test, cat_y_pred))
#print(average_precision_score(y_test, model.predict_proba(X_test)[:,1]))

Мальчики:
CatBoost: 
 accuracy = 0.5946601941747572 
 precision = 0.5863453815261044 
 recall = 0.6952380952380952 
 f1 = 0.6361655773420479
0.5926685525695427


In [45]:
#Random Forest 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=19, random_state=42)
clf.fit(X_train, y_train)
tree_y_pred = clf.predict(X_test)
RF_accuracy, RF_precision = accuracy_score(y_test, tree_y_pred), precision_score(y_test, tree_y_pred) 
RF_recall, RF_f1 = recall_score(y_test, tree_y_pred), f1_score(y_test, tree_y_pred)
print('Мальчики:')
print('CatBoost: \n accuracy = {} \n precision = {} \n recall = {} \n f1 = {}'.format(RF_accuracy, RF_precision, RF_recall, RF_f1))
print(roc_auc_score(y_test, tree_y_pred))

Мальчики:
CatBoost: 
 accuracy = 0.5703883495145631 
 precision = 0.569620253164557 
 recall = 0.6428571428571429 
 f1 = 0.6040268456375839
0.5689533239038189


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [46]:
#SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel = 'linear'))
clf2.fit(X_train, y_train)
svm_y_pred = clf2.predict(X_test)
SV_accuracy, SV_precision = accuracy_score(y_test, svm_y_pred), precision_score(y_test, svm_y_pred) 
SV_recall, SV_f1 = recall_score(y_test, svm_y_pred), f1_score(y_test, svm_y_pred)
print('Мальчики:')
print('SVM: \n accuracy = {} \n precision = {} \n recall = {} \n f1 = {}'.format(SV_accuracy, SV_precision, SV_recall, SV_f1))
print(roc_auc_score(y_test, svm_y_pred))

Мальчики:
SVM: 
 accuracy = 0.5703883495145631 
 precision = 0.5921787709497207 
 recall = 0.5047619047619047 
 f1 = 0.544987146529563
0.5716878830740217


In [47]:
y_test.value_counts()

1    210
0    202
Name: class, dtype: int64

In [48]:
pd.Series(cat_y_pred).value_counts()

1    249
0    163
dtype: int64

In [49]:
roc_auc_score(y_test, cat_y_pred)

0.5926685525695427

In [50]:
#LOGISTIC REGRESSION
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
LR_accuracy, LR_precision = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred) 
LR_recall, LR_f1 = recall_score(y_test, y_pred), f1_score(y_test, y_pred)
print('Мальчики:')
print('Логистическая регрессия:\n accuracy = {} \n precision = {} \n recall = {} \n f1 = {}'.format(LR_accuracy, LR_precision, LR_recall, LR_f1))
print(roc_auc_score(y_test, y_pred))

Мальчики:
Логистическая регрессия:
 accuracy = 0.5849514563106796 
 precision = 0.5802469135802469 
 recall = 0.6714285714285714 
 f1 = 0.6225165562913907
0.5832390381895332


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
